In [1]:
import numpy as np
import pandas as pd
import math
import os
from utils import *
from sklearn.model_selection import train_test_split
from models import *
from grid_search import *

# df_test = pd.read_csv("data/sampleSubmission.csv")
# args = read_config('config_models.yaml')
# model = SVDplusplus(args)
# print(args.svdpp)
# train, test = train_test_split(df, test_size=args.test_set_size, random_state=args.random_state)
# model.train(df_train)
# model.predict(df_test, output_file="output.csv")


In [2]:
df_train = pd.read_csv("data/data_train.csv")
df_test = pd.read_csv("data/sampleSubmission.csv")
args = read_config('config_models.yaml')

model = NeuralCF(args)

model.train(df_train)
model.predict(df_test, output_file='ncf_output.csv')







INFO:recommenders.models.ncf.dataset:Indexing temp.csv ...
/Users/oliver/mambaforge/envs/ci_project_3_9/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2024-05-10 00:12:26.736392: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


KeyboardInterrupt: 

In [ ]:
df_train = pd.read_csv("data/data_train.csv")
best_params, best_score = perform_grid_search("NeuralCF", df_train, 'grid_search_args.yaml')
print(best_params)
print(best_score)

In [ ]:

# Model parameters
EPOCHS = 50
BATCH_SIZE = 256
SEED = 42

In [ ]:
model = NCF (
    n_users=10000, 
    n_items=1000,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

In [ ]:
def preprocess(arr, n_row, n_col, imputation):
    ### Column Normalization
    masked = np.ma.masked_equal(arr, 0)
    # to check: mean along row / col have effects on results?
    mean_cols = np.tile(np.ma.mean(masked, axis=0).data, (n_row, 1))
    std_cols = np.tile(np.ma.std(masked, axis=0).data, (n_row, 1))
    normalized_arr = ((masked - mean_cols) / std_cols).data

    ### Imputation
    if imputation == "zero":
        imputed_arr = normalized_arr
    elif imputation == "mean":
        imputed_arr = mean_cols * (normalized_arr == 0) + arr * (normalized_arr != 0)

    return imputed_arr, mean_cols, std_cols

In [ ]:
from recommenders.models.ncf.dataset import Dataset as NCFDataset

df_train = pd.read_csv("data/data_train.csv")
users, movies, predictions = extract_users_items_predictions(df_train)

data_matrix = create_data_matrix(users, movies, predictions)

imp_arr, mean, std = preprocess(data_matrix, 10000, 1000, "zero")
print(mean)
print(std)

predictions_real = imp_arr[users, movies]


df_new = pd.DataFrame({
    'itemID': movies,
    'userID': users,
    'rating': predictions_real
})
df_new.sort_values(by=['userID'], inplace=True)




print(df_new)
df_new.to_csv('testing.csv')

data = NCFDataset(train_file='testing.csv', test_file=None, seed=42, binary=False,
                          overwrite_test_file_full=False, n_neg_test=0, n_neg=0)

model.fit(data)

print(model.predict(100,100))
print(model.predict(200,200))
print(model.predict(300,300))
print(model.predict(400,400))


In [ ]:
def postprocess(raw_predictions, data_mean = None, data_std = None, min_rate = 1, max_rate = 5, denorm = True):
    denormalized_predictions = raw_predictions
    if denorm:
        denormalized_predictions = raw_predictions * data_std + data_mean
    clipped_predicted = np.clip(denormalized_predictions, min_rate, max_rate)
    return clipped_predicted


In [ ]:
# import itertools
# predictions = [[row, col, model.predict(row, col)] for (row, col) in itertools.product(np.arange(10000), np.arange(1000))]

# predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
# print(1)
# reconstructed = np.zeros((10000, 1000))
# all_pred = predictions.copy(deep=True)
# print(2)
# for ind, row in all_pred.iterrows():
#     reconstructed[int(row.userID), int(row.itemID)] = row.prediction
# print(3)
reconstructed = np.zeros((10000, 1000))
for i in range(10000):
    if i % 100 == 0:
        print("i")
    for j in range(1000):
        reconstructed[i,j] = model.predict(i,j)
print(3)
reconstructed = postprocess(reconstructed, mean, std)
print(4)
df_test = pd.read_csv("data/sampleSubmission.csv")
users, movies, _ = extract_users_items_predictions(df_test)
print(5)
create_submission_from_matrix(reconstructed, users, movies, store_path="subnetmissionwork.csv")

print(model.predict(100,100))
print(model.predict(200,200))
print(model.predict(300,300))
print(model.predict(400,400))

